# INSTALAÇÕES NECESSÁRIAS

In [1]:
# instalação do pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# instalação da biblioteca google cloud storage file system
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# instalando pandera
!pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# IMPORTANDO BIBLIOTECAS

In [4]:
# importando bibliotecas 
from pyspark.sql import SparkSession
from google.cloud import storage
import os
import pandera as pa
import pandas as pd
import numpy as np

# importando as functions do spark
import pyspark.sql.functions as F
# importando nulltype do spark
from pyspark.sql.types import NullType

# CONFIGURAÇÃO GCS

In [5]:
# configuração da chave de segurança
serviceAccount = '/content/drive/MyDrive/aulas-bc26-giovana-1320ca3f36cc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [6]:
# fazer acesso a bucket
client = storage.Client()

# criar uma variável
bucket = client.get_bucket('aulas-bc26-dados-gbs')

# escolher arquivo dentro da bucket
bucket.blob('/spotify/spotify.csv')

# criar uma variável para receber o caminho do arquivo
path = 'gs://aulas-bc26-dados-gbs/spotify/spotify.csv'

In [7]:
# criando df do pandas pelo gcs
# df_pandas = pd.read_csv(path)
# df_pandas.head()

# CONFIGURANDO SPARKSESSION

In [8]:
# criando a sparksession com um config adicional para conexão spark com o cloud storage
spark = (
    SparkSession.builder
                .master('local')
                .appName('gcsfs')
                .config('spark.ui.port', '4050')
                .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                .getOrCreate()
)

In [9]:
spark

# CRIANDO DF

In [10]:
# ler o arquivo csv que está dentro da bucket
df = ( spark.read.format('csv')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .option('delimiter', ',')
                 .load(path)
)

In [11]:
df.show(5, truncate=False)

+---+--------------------------+---------------+----------------+----------------------+------------+------------------+----------------+----+--------+-------------------+----------+-----------+-------+--------------+-------------+
|_c0|song_name                 |song_popularity|song_duration_ms|acousticness          |danceability|energy            |instrumentalness|key |liveness|loudness           |audio_mode|speechiness|tempo  |time_signature|audio_valence|
+---+--------------------------+---------------+----------------+----------------------+------------+------------------+----------------+----+--------+-------------------+----------+-----------+-------+--------------+-------------+
|0  |Boulevard of Broken Dreams|73             |262333          |0.005520000000000001kg|0.496mol/L  |0.682             |2.94e-05        |8.0 |0.0589  |-4.095             |1         |0.0294     |167.06 |4             |0.474        |
|1  |In The End                |66             |216933          |0.0103k

In [12]:
# converter para um df do pandas
# df2 = df.toPandas()

In [13]:
# converter do pandas para o spark
# df_spark = spark.createDataFrame(df_pandas)

# ATIVIDADE SPARK - SPOTIFY

## 1) RENOMEAR TODAS AS COLUNAS (TRADUZINDO PARA O PORTUGUÊS)

In [14]:
# renomeando colunas
df = ( df.withColumnRenamed('_c0', 'id')
         .withColumnRenamed('song_name', 'nome_musica')
         .withColumnRenamed('song_popularity', 'popularidade')
         .withColumnRenamed('song_duration_ms', 'duracao')
         .withColumnRenamed('acousticness', 'acustica')
         .withColumnRenamed('danceability', 'danceabilidade')
         .withColumnRenamed('energy', 'energia')
         .withColumnRenamed('instrumentalness', 'instrumentalidade')
         .withColumnRenamed('key', 'nota_musical')
         .withColumnRenamed('liveness', 'ao_vivo')
         .withColumnRenamed('loudness', 'volume')
         .withColumnRenamed('audio_mode', 'modo_de_audio')
         .withColumnRenamed('speechiness', 'discurso')
         .withColumnRenamed('tempo', 'ritmo')
         .withColumnRenamed('time_signature', 'assinatura_de_tempo')
         .withColumnRenamed('audio_valence', 'valencia_de_audio')
)

In [15]:
df.show(truncate=False)

+---+--------------------------+------------+-------+----------------------+-----------------------+------------------+---------------------+------------+-------------------+-------------------+-------------+--------------------+-------+-------------------+------------------+
|id |nome_musica               |popularidade|duracao|acustica              |danceabilidade         |energia           |instrumentalidade    |nota_musical|ao_vivo            |volume             |modo_de_audio|discurso            |ritmo  |assinatura_de_tempo|valencia_de_audio |
+---+--------------------------+------------+-------+----------------------+-----------------------+------------------+---------------------+------------+-------------------+-------------------+-------------+--------------------+-------+-------------------+------------------+
|0  |Boulevard of Broken Dreams|73          |262333 |0.005520000000000001kg|0.496mol/L             |0.682             |2.94e-05             |8.0         |0.0589         

In [16]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- nome_musica: string (nullable = true)
 |-- popularidade: string (nullable = true)
 |-- duracao: string (nullable = true)
 |-- acustica: string (nullable = true)
 |-- danceabilidade: string (nullable = true)
 |-- energia: string (nullable = true)
 |-- instrumentalidade: string (nullable = true)
 |-- nota_musical: double (nullable = true)
 |-- ao_vivo: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- modo_de_audio: string (nullable = true)
 |-- discurso: string (nullable = true)
 |-- ritmo: string (nullable = true)
 |-- assinatura_de_tempo: string (nullable = true)
 |-- valencia_de_audio: double (nullable = true)



## 2) FAÇA UMA ANÁLISE INICIAL, ORDENANDO POR COLUNAS DE FORMA ASCENDENTE E DESCENDENTE

In [17]:
# análise de colunas por ordem crescente pelo nome_musica
df.orderBy(F.col('nome_musica').asc()).show()

+-----+--------------------+------------+-------+----------+--------------------+------------------+------------------+------------+-------------------+-------+-------------+--------+------------------+-------------------+-------------------+
|   id|         nome_musica|popularidade|duracao|  acustica|      danceabilidade|           energia| instrumentalidade|nota_musical|            ao_vivo| volume|modo_de_audio|discurso|             ritmo|assinatura_de_tempo|  valencia_de_audio|
+-----+--------------------+------------+-------+----------+--------------------+------------------+------------------+------------+-------------------+-------+-------------+--------+------------------+-------------------+-------------------+
|16726|"""40"" - Live Ve...|           1| 225440|   0.109kg|          0.257mol/L|             0.472|          0.000945|         8.0| 0.8290000000000001|-16.259|            1|  0.0399|           143.517|                  4|0.28600000000000003|
|14132|            """45"""|

In [18]:
# análise de colunas por ordem decrescente pelo nome_musica
df.orderBy(F.col('nome_musica').desc()).show()

+----+---------------------------------------+------------+-------+--------------------+--------------------+------------------+-----------------+------------+-------------------+-------------------+-------------+-------------------+------------------+-------------------+------------------+
|  id|                            nome_musica|popularidade|duracao|            acustica|      danceabilidade|           energia|instrumentalidade|nota_musical|            ao_vivo|             volume|modo_de_audio|           discurso|             ritmo|assinatura_de_tempo| valencia_de_audio|
+----+---------------------------------------+------------+-------+--------------------+--------------------+------------------+-----------------+------------+-------------------+-------------------+-------------+-------------------+------------------+-------------------+------------------+
|6684|                             피 땀 눈물|          50| 217051|            0.0244kg|          0.583mol/L|             0.885|

In [19]:
# análise de colunas por ordem crescente pela valencia_de_audio
df.orderBy(F.col('valencia_de_audio').asc()).show()

+-----+--------------------+------------+--------------------+--------------------+--------------------+-------------------+------------------+------------+-------------------+-------------------+-------------+--------------------+-----------------+-------------------+-----------------+
|   id|         nome_musica|popularidade|             duracao|            acustica|      danceabilidade|            energia| instrumentalidade|nota_musical|            ao_vivo|             volume|modo_de_audio|            discurso|            ritmo|assinatura_de_tempo|valencia_de_audio|
+-----+--------------------+------------+--------------------+--------------------+--------------------+-------------------+------------------+------------+-------------------+-------------------+-------------+--------------------+-----------------+-------------------+-----------------+
| 1757| Party In The U.S.A.|     nao_sei|0.8220000000000001kg|          0.519mol/L|                0.36|                0.0|            

In [20]:
# análise de colunas por ordem decrescente pela valencia_de_audio
df.orderBy(F.col('valencia_de_audio').desc()).show()

+-----+--------------------+------------+-------+--------------------+--------------------+--------------------+------------------+------------+------------------+------------------+------------------+--------+------------------+-------------------+-----------------+
|   id|         nome_musica|popularidade|duracao|            acustica|      danceabilidade|             energia| instrumentalidade|nota_musical|           ao_vivo|            volume|     modo_de_audio|discurso|             ritmo|assinatura_de_tempo|valencia_de_audio|
+-----+--------------------+------------+-------+--------------------+--------------------+--------------------+------------------+------------+------------------+------------------+------------------+--------+------------------+-------------------+-----------------+
|10528|"Juntos (Together...|      USA"""|     44|              200466|            0.0986kg|0.707000000000000...|             0.866|         0.0|                10|             0.341|-6.04799999999

## 3) VERIFIQUE SE HÁ ALGUMA COLUNA QUE NÃO TENHA INFORMAÇÃO RELEVANTE E FAÇA UM DROP

In [21]:
# removendo colunas id para análise de duplicações
df2 = df.drop(F.col('id'))
df2.show()

+--------------------+------------+-------+--------------------+--------------------+------------------+--------------------+------------+-------------------+-------------------+-------------+--------------------+-------+-------------------+------------------+
|         nome_musica|popularidade|duracao|            acustica|      danceabilidade|           energia|   instrumentalidade|nota_musical|            ao_vivo|             volume|modo_de_audio|            discurso|  ritmo|assinatura_de_tempo| valencia_de_audio|
+--------------------+------------+-------+--------------------+--------------------+------------------+--------------------+------------+-------------------+-------------------+-------------+--------------------+-------+-------------------+------------------+
|Boulevard of Brok...|          73| 262333|0.005520000000000...|          0.496mol/L|             0.682|            2.94e-05|         8.0|             0.0589|             -4.095|            1|              0.0294| 167

## 4) VERIFIQUE SE HÁ LINHAS DUPLICADAS (DUPLICATAS) E REMOVA-AS

In [22]:
# analisando quantas linhas duplicadas temos
total_linhas = df2.count()
total_linhas_distintas = df2.distinct().count()

print(total_linhas - total_linhas_distintas)

3903


In [23]:
# removendo linhas duplicadas
df3 = df2.dropDuplicates()
total_linhas = df3.count()
total_linhas_distintas = df3.distinct().count()

print(total_linhas - total_linhas_distintas)

0


## 5) VERIFIQUE SE HÁ DADOS INCONSISTENTES (Dica: Unidades de medida)

In [24]:
df3.printSchema()

root
 |-- nome_musica: string (nullable = true)
 |-- popularidade: string (nullable = true)
 |-- duracao: string (nullable = true)
 |-- acustica: string (nullable = true)
 |-- danceabilidade: string (nullable = true)
 |-- energia: string (nullable = true)
 |-- instrumentalidade: string (nullable = true)
 |-- nota_musical: double (nullable = true)
 |-- ao_vivo: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- modo_de_audio: string (nullable = true)
 |-- discurso: string (nullable = true)
 |-- ritmo: string (nullable = true)
 |-- assinatura_de_tempo: string (nullable = true)
 |-- valencia_de_audio: double (nullable = true)



In [25]:
# inconsistências encontradas: aspas nos nomes 
df3.select('nome_musica').distinct().orderBy(F.col('nome_musica').asc()).show(truncate=False)

+---------------------------------------------------------------------------------------+
|nome_musica                                                                            |
+---------------------------------------------------------------------------------------+
|"""40"" - Live Version 1983"                                                           |
|"""45"""                                                                               |
|"""99"""                                                                               |
|"""Heroes"" - Single Version; 2014 Remastered Version"                                 |
|"All My Love is ""Free"""                                                              |
|"Any Way You Want It (World Championship Medley) - From ""Pitch Perfect 2"" Soundtrack"|
|"Back To Basics - From ""Pitch Perfect 2"" Soundtrack"                                 |
|"Baila - from the film ""Music of the Heart"""                                         |
|"CAN'T ST

In [26]:
# inconsistências encontradas: USA"""
df3.select('popularidade').distinct().orderBy(F.col('popularidade').asc()).show(truncate=False)

+------------+
|popularidade|
+------------+
| USA"""     |
|0           |
|1           |
|10          |
|100         |
|11          |
|12          |
|13          |
|14          |
|15          |
|16          |
|17          |
|18          |
|19          |
|2           |
|20          |
|21          |
|22          |
|23          |
|24          |
+------------+
only showing top 20 rows



In [27]:
# inconsistências encontradas: -18528908788662, 0.8220000000000001kg, 44
df3.select('duracao').distinct().orderBy(F.col('duracao').asc()).show(truncate=False)

+--------------------+
|duracao             |
+--------------------+
|-18528908788662     |
|0.8220000000000001kg|
|100026              |
|100242              |
|100465              |
|100571              |
|100629              |
|101053              |
|101270              |
|102077              |
|102170              |
|102243              |
|102269              |
|102400              |
|102536              |
|102560              |
|102620              |
|102661              |
|102791              |
|102857              |
+--------------------+
only showing top 20 rows



In [28]:
# inconsistências encontradas: unidade de medida
df3.select('acustica').distinct().orderBy(F.col('acustica').asc()).show(truncate=False)

+----------+
|acustica  |
+----------+
|0.000101kg|
|0.000102kg|
|0.000103kg|
|0.000105kg|
|0.000106kg|
|0.000107kg|
|0.000108kg|
|0.000109kg|
|0.000111kg|
|0.000112kg|
|0.000113kg|
|0.000114kg|
|0.000115kg|
|0.000116kg|
|0.000118kg|
|0.000119kg|
|0.00011kg |
|0.000121kg|
|0.000122kg|
|0.000123kg|
+----------+
only showing top 20 rows



In [29]:
# inconsistências encontradas: 0.0986kg e unidade de medida
df3.select('danceabilidade').distinct().orderBy(F.col('danceabilidade').asc()).show(truncate=False)

+--------------+
|danceabilidade|
+--------------+
|0.0594mol/L   |
|0.0617mol/L   |
|0.0625mol/L   |
|0.066mol/L    |
|0.0674mol/L   |
|0.0684mol/L   |
|0.0722mol/L   |
|0.0812mol/L   |
|0.0819mol/L   |
|0.081mol/L    |
|0.0833mol/L   |
|0.0855mol/L   |
|0.0868mol/L   |
|0.0874mol/L   |
|0.0962mol/L   |
|0.0986kg      |
|0.0mol/L      |
|0.101mol/L    |
|0.102mol/L    |
|0.105mol/L    |
+--------------+
only showing top 20 rows



In [30]:
# inconsistências encontradas: nao_sei
df3.select('energia').distinct().orderBy(F.col('energia').desc()).show(truncate=False)

+-------+
|energia|
+-------+
|nao_sei|
|0.999  |
|0.997  |
|0.996  |
|0.995  |
|0.994  |
|0.993  |
|0.992  |
|0.991  |
|0.99   |
|0.989  |
|0.988  |
|0.987  |
|0.986  |
|0.985  |
|0.984  |
|0.983  |
|0.982  |
|0.981  |
|0.98   |
+-------+
only showing top 20 rows



In [31]:
# inconsistências encontradas: nao_sei
df3.select('instrumentalidade').distinct().orderBy(F.col('instrumentalidade').desc()).show(truncate=False)

+---------------------+
|instrumentalidade    |
+---------------------+
|nao_sei              |
|9e-06                |
|9e-05                |
|9.990000000000003e-05|
|9.98e-06             |
|9.98e-05             |
|9.97e-05             |
|9.96e-05             |
|9.94e-06             |
|9.940000000000003e-05|
|9.93e-06             |
|9.92e-05             |
|9.8e-06              |
|9.8e-05              |
|9.88e-05             |
|9.86e-06             |
|9.86e-05             |
|9.85e-05             |
|9.84e-06             |
|9.84e-05             |
+---------------------+
only showing top 20 rows



In [32]:
# inconsistências encontradas: nao_sei, -8.575
df3.select('ao_vivo').distinct().orderBy(F.col('ao_vivo').desc()).show(truncate=False)

+-------+
|ao_vivo|
+-------+
|nao_sei|
|10     |
|0.986  |
|0.984  |
|0.983  |
|0.981  |
|0.979  |
|0.978  |
|0.977  |
|0.975  |
|0.974  |
|0.967  |
|0.961  |
|0.956  |
|0.952  |
|0.95   |
|0.949  |
|0.948  |
|0.944  |
|0.941  |
+-------+
only showing top 20 rows



In [33]:
# inconsistências encontradas: nao_sei, valores positivos
df3.select('volume').distinct().orderBy(F.col('volume').desc()).show(truncate=False)

+--------------------+
|volume              |
+--------------------+
|nao_sei             |
|1.585               |
|1.342               |
|0.878               |
|0.525               |
|0.341               |
|0.198               |
|0.119               |
|0.052000000000000005|
|0                   |
|-9.999              |
|-9.997              |
|-9.996              |
|-9.995              |
|-9.993              |
|-9.992              |
|-9.991              |
|-9.99               |
|-9.989              |
|-9.988              |
+--------------------+
only showing top 20 rows



In [34]:
# inconsistências encontradas: -6.047999999999999, 0.105, nao_sei
df3.select('modo_de_audio').distinct().orderBy(F.col('modo_de_audio').asc()).show(truncate=False)

+------------------+
|modo_de_audio     |
+------------------+
|-6.047999999999999|
|0                 |
|0.105             |
|1                 |
|nao_sei           |
+------------------+



In [35]:
# inconsistências encontradas: 0.nao_sei, 97.42
df3.select('discurso').distinct().orderBy(F.col('discurso').desc()).show(truncate=False)

+------------------+
|discurso          |
+------------------+
|97.42             |
|0.nao_sei         |
|0.941             |
|0.94              |
|0.936             |
|0.915             |
|0.906             |
|0.894             |
|0.8909999999999999|
|0.89              |
|0.8690000000000001|
|0.831             |
|0.83              |
|0.826             |
|0.821             |
|0.8079999999999999|
|0.797             |
|0.794             |
|0.7909999999999999|
|0.759             |
+------------------+
only showing top 20 rows



In [36]:
# inconsistências encontradas: nao_sei, 0.0, 0.0324
df3.select('ritmo').distinct().orderBy(F.col('ritmo').asc()).show(truncate=False)

+-------+
|ritmo  |
+-------+
|0.0    |
|0.0324 |
|100.0  |
|100.001|
|100.002|
|100.003|
|100.004|
|100.005|
|100.006|
|100.007|
|100.008|
|100.01 |
|100.011|
|100.012|
|100.013|
|100.014|
|100.015|
|100.016|
|100.017|
|100.018|
+-------+
only showing top 20 rows



In [37]:
# inconsistências encontradas: nao_sei, 2800000000, 97.01, 0
df3.select('assinatura_de_tempo').distinct().orderBy(F.col('assinatura_de_tempo').asc()).show(truncate=False)

+-------------------+
|assinatura_de_tempo|
+-------------------+
|0                  |
|0.7                |
|1                  |
|2800000000         |
|3                  |
|4                  |
|5                  |
|97.01              |
|nao_sei            |
+-------------------+



In [38]:
df3.groupBy('assinatura_de_tempo').count().orderBy(F.col('assinatura_de_tempo').asc()).show()

+-------------------+-----+
|assinatura_de_tempo|count|
+-------------------+-----+
|                  0|    3|
|                0.7|    1|
|                  1|   67|
|         2800000000|    1|
|                  3|  684|
|                  4|13979|
|                  5|  195|
|              97.01|    1|
|            nao_sei|    1|
+-------------------+-----+



In [39]:
# inconsistências encontradas: null, 4.0
df3.select('valencia_de_audio').distinct().orderBy(F.col('valencia_de_audio').asc()).show(truncate=False)

+-----------------+
|valencia_de_audio|
+-----------------+
|null             |
|0.0              |
|0.023            |
|0.0243           |
|0.0277           |
|0.0292           |
|0.0301           |
|0.0309           |
|0.0312           |
|0.0316           |
|0.032            |
|0.0321           |
|0.0326           |
|0.0329           |
|0.033            |
|0.0331           |
|0.0333           |
|0.0336           |
|0.0337           |
|0.0338           |
+-----------------+
only showing top 20 rows



## 6) REMOVA AS INCONSISTÊNCIAS

### tratamento de inconsistências: aspas

In [40]:
df3.filter(F.col('nome_musica').contains('"')).show()

+--------------------+------------+-------+---------+--------------------+--------------------+-----------------+------------+-------------------+-------------------+------------------+--------+------------------+-------------------+------------------+
|         nome_musica|popularidade|duracao| acustica|      danceabilidade|             energia|instrumentalidade|nota_musical|            ao_vivo|             volume|     modo_de_audio|discurso|             ritmo|assinatura_de_tempo| valencia_de_audio|
+--------------------+------------+-------+---------+--------------------+--------------------+-----------------+------------+-------------------+-------------------+------------------+--------+------------------+-------------------+------------------+
|"Dusk Till Dawn -...|          61| 202412|0.00136kg|0.655999999999999...|               0.799|         0.000325|        11.0|              0.131|             -4.342|                 0|  0.0405|           115.936|                  4|        

In [41]:
# substituindo parte da string com outra string
from pyspark.sql.functions import regexp_replace
df4 = df3.withColumn('nome_musica', regexp_replace('nome_musica', '"', ''))

In [42]:
df4.filter(F.col('nome_musica').contains('"')).show()

+-----------+------------+-------+--------+--------------+-------+-----------------+------------+-------+------+-------------+--------+-----+-------------------+-----------------+
|nome_musica|popularidade|duracao|acustica|danceabilidade|energia|instrumentalidade|nota_musical|ao_vivo|volume|modo_de_audio|discurso|ritmo|assinatura_de_tempo|valencia_de_audio|
+-----------+------------+-------+--------+--------------+-------+-----------------+------------+-------+------+-------------+--------+-----+-------------------+-----------------+
+-----------+------------+-------+--------+--------------+-------+-----------------+------------+-------+------+-------------+--------+-----+-------------------+-----------------+



### tratamento de inconsistências: unidades de medida

In [43]:
# substituindo parte da string com outra string
df5 = df4.withColumn('acustica', regexp_replace('acustica', 'kg', ''))
df5.show(truncate=False)

+------------------------------+------------+-------+----------------------+-----------------------+------------------+---------------------+------------+-------------------+------------------+-------------+-------------------+-------+-------------------+------------------+
|nome_musica                   |popularidade|duracao|acustica              |danceabilidade         |energia           |instrumentalidade    |nota_musical|ao_vivo            |volume            |modo_de_audio|discurso           |ritmo  |assinatura_de_tempo|valencia_de_audio |
+------------------------------+------------+-------+----------------------+-----------------------+------------------+---------------------+------------+-------------------+------------------+-------------+-------------------+-------+-------------------+------------------+
|Yeah!                         |79          |250373 |0.0183                |0.894mol/L             |0.7909999999999999|0.0                  |2.0         |0.0388             |-

In [44]:
# substituindo parte da string com outra string
df6 = df5.withColumn('danceabilidade', regexp_replace('danceabilidade', 'mol/L', ''))
df6.show(truncate=False)

+------------------------------+------------+-------+----------------------+------------------+------------------+---------------------+------------+-------------------+------------------+-------------+-------------------+-------+-------------------+------------------+
|nome_musica                   |popularidade|duracao|acustica              |danceabilidade    |energia           |instrumentalidade    |nota_musical|ao_vivo            |volume            |modo_de_audio|discurso           |ritmo  |assinatura_de_tempo|valencia_de_audio |
+------------------------------+------------+-------+----------------------+------------------+------------------+---------------------+------------+-------------------+------------------+-------------+-------------------+-------+-------------------+------------------+
|Yeah!                         |79          |250373 |0.0183                |0.894             |0.7909999999999999|0.0                  |2.0         |0.0388             |-4.699            |1 

## 7) VERIFIQUE OS TIPOS DE DADOS DAS COLUNAS, E COLOQUE PARA UM TIPO MAIS ADEQUADO

In [45]:
df6.printSchema()

root
 |-- nome_musica: string (nullable = true)
 |-- popularidade: string (nullable = true)
 |-- duracao: string (nullable = true)
 |-- acustica: string (nullable = true)
 |-- danceabilidade: string (nullable = true)
 |-- energia: string (nullable = true)
 |-- instrumentalidade: string (nullable = true)
 |-- nota_musical: double (nullable = true)
 |-- ao_vivo: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- modo_de_audio: string (nullable = true)
 |-- discurso: string (nullable = true)
 |-- ritmo: string (nullable = true)
 |-- assinatura_de_tempo: string (nullable = true)
 |-- valencia_de_audio: double (nullable = true)



In [46]:
# tranformando tipos com o cast
df6 = (df6.withColumn("nome_musica", df6["nome_musica"].cast('string'))
          .withColumn("popularidade", df6["popularidade"].cast('int'))
          .withColumn("duracao", df6["duracao"].cast('int'))
          .withColumn("acustica", df6["acustica"].cast('float'))
          .withColumn("danceabilidade", df6["danceabilidade"].cast('float'))
          .withColumn("energia", df6["energia"].cast('float'))
          .withColumn("nota_musical", df6["nota_musical"].cast('int'))
          .withColumn("ao_vivo", df6["ao_vivo"].cast('float'))
          .withColumn("volume", df6["volume"].cast('float'))
          .withColumn("modo_de_audio", df6["modo_de_audio"].cast('int'))
          .withColumn("discurso", df6["discurso"].cast('float'))
          .withColumn("ritmo", df6["ritmo"].cast('float'))
          .withColumn("assinatura_de_tempo", df6["assinatura_de_tempo"].cast('float'))
          .withColumn("valencia_de_audio", df6["valencia_de_audio"].cast('float'))
)
df6.printSchema()

root
 |-- nome_musica: string (nullable = true)
 |-- popularidade: integer (nullable = true)
 |-- duracao: integer (nullable = true)
 |-- acustica: float (nullable = true)
 |-- danceabilidade: float (nullable = true)
 |-- energia: float (nullable = true)
 |-- instrumentalidade: string (nullable = true)
 |-- nota_musical: integer (nullable = true)
 |-- ao_vivo: float (nullable = true)
 |-- volume: float (nullable = true)
 |-- modo_de_audio: integer (nullable = true)
 |-- discurso: float (nullable = true)
 |-- ritmo: float (nullable = true)
 |-- assinatura_de_tempo: float (nullable = true)
 |-- valencia_de_audio: float (nullable = true)



## 8) FAÇA OUTRAS VERIFICAÇÕES DE ACORDO COM A SUA EXPERIÊNCIA EM PANDAS

In [47]:
# convertendo para um df do pandas
df_pandas = df6.toPandas()

In [48]:
df_pandas

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
0,Yeah!,79.0,250373.0,0.0183,0.894,0.791,0.0,2,0.0388,-4.699,1.0,0.1120,105.017998,4.0,0.583
1,Someday - Remastered,63.0,243333.0,0.0108,0.743,0.670,0.0702,1,0.2200,-4.547,0.0,0.0293,110.832001,4.0,0.934
2,Easy,58.0,184141.0,0.3550,0.847,0.703,3.950000000000001e-05,6,0.0938,-4.434,1.0,0.0746,112.044998,4.0,0.777
3,Do You Wanna Party,56.0,201960.0,0.1230,0.794,0.844,3.16e-06,11,0.0988,-3.222,1.0,0.0904,119.988998,4.0,0.794
4,La Media Vuelta,72.0,161040.0,0.2910,0.631,0.683,0.0,7,0.0994,-8.788,1.0,0.0344,97.959000,4.0,0.708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14927,Anywhere,77.0,205250.0,0.0162,0.717,0.872,1.99e-06,11,0.2050,-3.279,1.0,0.0357,120.026001,4.0,0.610
14928,Real Rich (feat. Gucci Mane),74.0,124525.0,0.0353,0.947,0.441,0.0,2,0.1110,-9.311,1.0,0.1350,125.010002,4.0,0.275
14929,I Found,58.0,274226.0,0.8770,0.280,0.317,0.007140000000000001,7,0.1080,-12.433,0.0,0.0468,186.643997,3.0,0.237
14930,3:15,76.0,167018.0,0.3850,0.651,0.654,0.0,1,0.1810,-5.481,1.0,0.0466,141.975006,4.0,0.539


In [49]:
df_pandas.isnull().sum()

nome_musica            0
popularidade           2
duracao                2
acustica               1
danceabilidade         1
energia                2
instrumentalidade      0
nota_musical           0
ao_vivo                4
volume                 1
modo_de_audio          1
discurso               1
ritmo                  1
assinatura_de_tempo    1
valencia_de_audio      1
dtype: int64

### FILTROS NULOS

In [50]:
filtronulo1 = df_pandas.popularidade.isna()
df_pandas.loc[filtronulo1]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
385,Party In The U.S.A.,NaN,NaN,NaN,0.36,0.0,10,0,-8.575,0.000,0.0,97.419998,4.0000,0.700000,NaN
427,Juntos (Together) - From McFarland,NaN,44.0,200466.0,NaN,NaN,0.866,0,10.000,0.341,-6.0,0.000000,0.0324,97.010002,4.0


In [51]:
filtronulo2 = df_pandas.duracao.isna()
df_pandas.loc[filtronulo2]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
385,Party In The U.S.A.,NaN,NaN,NaN,0.36,0.000,10,0,-8.5750,0.00,0.0,97.419998,4.000000,0.7,NaN
10726,White Boi,59.0,NaN,0.0983,0.83,0.497,0.0,1,0.0906,-6.94,1.0,0.052000,127.964996,4.0,0.634


In [52]:
filtronulo3 = df_pandas.energia.isna()
df_pandas.loc[filtronulo3]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
427,Juntos (Together) - From McFarland,NaN,44.0,200466.000,NaN,NaN,0.866,0,10.000,0.341,-6.0,0.00,0.032400,97.010002,4.000
8384,Banana Clip - Spanish Version,57.0,202373.0,0.107,0.398,NaN,0.0,1,0.121,-5.626,0.0,0.12,83.828003,4.000000,0.719


In [53]:
filtronulo4 = df_pandas.ao_vivo.isna()
df_pandas.loc[filtronulo4]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
3419,Blister in the Sun - 2002 Remastered Version,69.0,144333.0,0.219000,0.701,0.261,0.0,0,NaN,-19.846001,1.0,0.0938,97.042000,4.0,0.890
3440,From Out of Nowhere,36.0,139293.0,0.000004,0.321,0.929,0.925,2,NaN,-4.367000,1.0,0.0584,96.250000,3.0,0.309
4165,Break My Stride,62.0,180240.0,0.200000,0.901,0.654,0.0,10,NaN,-10.150000,1.0,0.0655,110.130997,4.0,0.781
7350,Live Your Life (feat. Rihanna),76.0,338853.0,0.071000,0.375,0.862,0.0,11,NaN,-3.363000,0.0,0.2550,159.841003,4.0,0.478


In [54]:
filtronulo5 = df_pandas.volume.isna()
df_pandas.loc[filtronulo5]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
8086,Issa Goal,50.0,216774.0,0.161,0.708,0.767,0.0,4,0.328,NaN,0.0,0.258,124.453003,4.0,0.853


In [55]:
filtronulo6 = df_pandas.modo_de_audio.isna()
df_pandas.loc[filtronulo6]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
4192,Lhhc,48.0,189985.0,0.00732,0.629,0.993,0.0,7,0.148,-1.277,NaN,0.214,137.022995,4.0,0.329


In [56]:
filtronulo7 = df_pandas.discurso.isna()
df_pandas.loc[filtronulo7]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
13631,Keep on Loving You - Remastered,71.0,200293.0,0.0519,0.306,0.722,0.0,0,0.117,-5.963,1.0,NaN,174.731995,4.0,0.41


In [57]:
filtronulo8 = df_pandas.ritmo.isna()
df_pandas.loc[filtronulo8]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
5579,Rude (Acoustic),74.0,223854.0,0.358,0.736,0.847,0.0,1,0.101,-4.287,1.0,0.0756,NaN,4.0,0.915


In [58]:
filtronulo9 = df_pandas.assinatura_de_tempo.isna()
df_pandas.loc[filtronulo9] 

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
6791,Nobody's Fool,16.0,287400.0,0.0478,0.367,0.445,0.00284,9,0.199,-13.615,0.0,0.0286,67.196999,NaN,0.149


### tratamento de inconsistências: músicas com nomes e dados inconsistentes

In [59]:
# os dados estão muito inconsistêntes, então darei drop
fta = df_pandas.nome_musica == '-6.751'
df_pandas.loc[fta]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


In [60]:
df_pandas.drop([8086], axis=0, inplace=True)

In [61]:
df_pandas.loc[fta]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


In [62]:
# os dados estão muito inconsistêntes, então darei drop
ftb = df_pandas.nome_musica == '211851.0'
df_pandas.loc[ftb]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


In [63]:
df_pandas.drop([10726], axis=0, inplace=True)

In [64]:
df_pandas.loc[ftb]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


In [65]:
# os dados estão muito inconsistêntes, então darei drop
ftc = df_pandas.nome_musica == '4.0'
df_pandas.loc[ftc]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


In [66]:
df_pandas.drop([6791], axis=0, inplace=True)

In [67]:
df_pandas.loc[ftc]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


In [68]:
# os dados estão muito inconsistêntes, então darei drop
ftd = df_pandas.nome_musica == '1.0'
df_pandas.loc[ftd]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


In [69]:
df_pandas.drop([4192], axis=0, inplace=True)

In [70]:
df_pandas.loc[ftd]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


### tratamento de inconsistências: músicas com valores NaN

#### Party In The U.S.A.

In [71]:
# foi visto que a música Party In The U.S.A. tem mais 3 entradas, drop das antigas e deixamos a mais atual
ft1 = df_pandas.nome_musica == 'Party In The U.S.A.'
df_pandas.loc[ft1]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
385,Party In The U.S.A.,NaN,NaN,NaN,0.360,0.000,10,0,-8.5750,0.000,0.0,97.419998,4.000000,0.7,NaN
1995,Party In The U.S.A.,79.0,202066.0,0.00112,0.652,0.698,0.000115,10,0.0886,-4.667,0.0,0.042000,96.021004,4.0,0.470
9011,Party In The U.S.A.,76.0,202066.0,0.00124,0.454,0.708,0.000181,10,0.0679,-4.526,0.0,0.067500,192.179993,4.0,0.483


In [72]:
df_pandas.drop([385, 1995], axis=0, inplace=True)

In [73]:
df_pandas.loc[ft1]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
9011,Party In The U.S.A.,76.0,202066.0,0.00124,0.454,0.708,0.000181,10,0.0679,-4.526,0.0,0.0675,192.179993,4.0,0.483


#### Juntos (Together) - From McFarland

In [74]:
# foi visto que a música Juntos (Together) - From McFarland está com a maioria dos valores nulos ou inconsistentes, 
# então a tiraremos da análise
ft2 = df_pandas.nome_musica == 'Juntos (Together) - From McFarland'
df_pandas.loc[ft2]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
427,Juntos (Together) - From McFarland,NaN,44.0,200466.0,NaN,NaN,0.866,0,10.0,0.341,-6.0,0.0,0.0324,97.010002,4.0


In [75]:
df_pandas.drop([427], axis=0, inplace=True)

In [76]:
df_pandas.loc[ft2]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


#### White Boi

In [77]:
# a música White Boi não apresenta duplicações e tem apenas um valor nulo, então não daremos drop
ft3 = df_pandas.nome_musica == 'White Boi'
df_pandas.loc[ft3]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


#### Banana Clip - Spanish Version

In [78]:
# a música Banana Clip - Spanish Version apresenta duplicação, 
# então daremos drop na linha que tem um valor nulo
ft4 = df_pandas.nome_musica == 'Banana Clip - Spanish Version'
df_pandas.loc[ft4]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
4919,Banana Clip - Spanish Version,57.0,202373.0,0.107,0.398,0.66,0.0,1,0.121,-5.626,0.0,0.12,83.828003,4.0,0.719
8384,Banana Clip - Spanish Version,57.0,202373.0,0.107,0.398,NaN,0.0,1,0.121,-5.626,0.0,0.12,83.828003,4.0,0.719


In [79]:
df_pandas.drop([8384], axis=0, inplace=True)

In [80]:
df_pandas.loc[ft4]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
4919,Banana Clip - Spanish Version,57.0,202373.0,0.107,0.398,0.66,0.0,1,0.121,-5.626,0.0,0.12,83.828003,4.0,0.719


#### Blister in the Sun - 2002 Remastered Version	

In [81]:
# a música Blister in the Sun - 2002 Remastered Version apresenta duplicação, 
# então daremos drop na linha que tem um valor nulo
ft5 = df_pandas.nome_musica == 'Blister in the Sun - 2002 Remastered Version'
df_pandas.loc[ft5]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
3419,Blister in the Sun - 2002 Remastered Version,69.0,144333.0,0.219,0.701,0.261,0.0,0,NaN,-19.846001,1.0,0.0938,97.042,4.0,0.89
13421,Blister in the Sun - 2002 Remastered Version,69.0,144333.0,0.219,0.701,0.261,0.0,0,0.0821,-19.846001,1.0,0.0938,97.042,4.0,0.89


In [82]:
df_pandas.drop([3419], axis=0, inplace=True)

In [83]:
df_pandas.loc[ft5]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
13421,Blister in the Sun - 2002 Remastered Version,69.0,144333.0,0.219,0.701,0.261,0.0,0,0.0821,-19.846001,1.0,0.0938,97.042,4.0,0.89


#### From Out of Nowhere

In [84]:
# a música From Out of Nowhere não apresenta duplicações e tem apenas um valor nulo, então não daremos drop
ft6 = df_pandas.nome_musica == 'From Out of Nowhere'
df_pandas.loc[ft6]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
3440,From Out of Nowhere,36.0,139293.0,0.000004,0.321,0.929,0.925,2,NaN,-4.367,1.0,0.0584,96.25,3.0,0.309


#### Break My Stride

In [85]:
# a música Break My Stride apresenta duplicação, 
# então daremos drop na linha que tem um valor nulo
ft7 = df_pandas.nome_musica == 'Break My Stride'
df_pandas.loc[ft7]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
4165,Break My Stride,62.0,180240.0,0.2,0.901,0.654,0.0,10,NaN,-10.15,1.0,0.0655,110.130997,4.0,0.781
11012,Break My Stride,62.0,180240.0,0.2,0.901,0.654,0.0,10,0.165,-10.15,1.0,0.0655,110.130997,4.0,0.781


In [86]:
df_pandas.drop([4165], axis=0, inplace=True)

In [87]:
df_pandas.loc[ft7]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
11012,Break My Stride,62.0,180240.0,0.2,0.901,0.654,0.0,10,0.165,-10.15,1.0,0.0655,110.130997,4.0,0.781


#### Live Your Life (feat. Rihanna)	

In [88]:
# a música Live Your Life (feat. Rihanna) apresenta duplicação, 
# então daremos drop na linha que tem um valor nulo
ft8 = df_pandas.nome_musica == 'Live Your Life (feat. Rihanna)'
df_pandas.loc[ft8]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
4197,Live Your Life (feat. Rihanna),76.0,338853.0,0.071,0.375,0.862,0.0,11,0.211,-3.363,0.0,0.255,159.841003,4.0,0.478
7350,Live Your Life (feat. Rihanna),76.0,338853.0,0.071,0.375,0.862,0.0,11,NaN,-3.363,0.0,0.255,159.841003,4.0,0.478


In [89]:
df_pandas.drop([7350], axis=0, inplace=True)

In [90]:
df_pandas.loc[ft8]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
4197,Live Your Life (feat. Rihanna),76.0,338853.0,0.071,0.375,0.862,0.0,11,0.211,-3.363,0.0,0.255,159.841003,4.0,0.478


#### Issa Goal

In [91]:
# a música Issa Goal não apresenta duplicações e tem apenas um valor nulo, então não daremos drop
ft9 = df_pandas.nome_musica == 'Issa Goal'
df_pandas.loc[ft9]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


#### Lhhc	

In [92]:
# a música Lhhc não apresenta duplicações e tem apenas um valor nulo, então não daremos drop
ft10 = df_pandas.nome_musica == 'Lhhc'
df_pandas.loc[ft10]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


#### Keep on Loving You - Remastered	

In [93]:
# a música Keep on Loving You - Remastered não apresenta duplicações e tem apenas um valor nulo, então não daremos drop
ft11 = df_pandas.nome_musica == 'Keep on Loving You - Remastered'
df_pandas.loc[ft11]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
13631,Keep on Loving You - Remastered,71.0,200293.0,0.0519,0.306,0.722,0.0,0,0.117,-5.963,1.0,NaN,174.731995,4.0,0.41


#### Rude (Acoustic)

In [94]:
# a música Rude (Acoustic) apresenta duplicação, 
# então daremos drop na linha que tem um valor nulo
ft12 = df_pandas.nome_musica == 'Rude (Acoustic)'
df_pandas.loc[ft12]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
5579,Rude (Acoustic),74.0,223854.0,0.358,0.736,0.847,0.0,1,0.101,-4.287,1.0,0.0756,NaN,4.0,0.915
8066,Rude (Acoustic),74.0,223854.0,0.358,0.736,0.847,0.0,1,0.101,-4.287,1.0,0.0756,143.996002,4.0,0.915


In [95]:
df_pandas.drop([5579], axis=0, inplace=True)

In [96]:
df_pandas.loc[ft12]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
8066,Rude (Acoustic),74.0,223854.0,0.358,0.736,0.847,0.0,1,0.101,-4.287,1.0,0.0756,143.996002,4.0,0.915


#### Nobody's Fool

In [97]:
# a música Nobody's Fool não apresenta duplicações e tem apenas um valor nulo, então não daremos drop
ft13 = df_pandas.nome_musica == "Nobody's Fool"
df_pandas.loc[ft13]

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio


In [98]:
df_pandas.isnull().sum()

nome_musica            0
popularidade           0
duracao                0
acustica               0
danceabilidade         0
energia                0
instrumentalidade      0
nota_musical           0
ao_vivo                1
volume                 0
modo_de_audio          0
discurso               1
ritmo                  0
assinatura_de_tempo    0
valencia_de_audio      0
dtype: int64

### TRATAR VALORES NULOS/VAZIOS

In [99]:
df_pandas.duracao.median()

211851.0

In [100]:
df_pandas.loc[df_pandas['duracao'].isna()] = 211851.0

In [101]:
df_pandas.ao_vivo.median()

0.122

In [102]:
df_pandas.loc[df_pandas['ao_vivo'].isna()] = 0.122

In [103]:
df_pandas.volume.median()

-6.751

In [104]:
df_pandas.loc[df_pandas['volume'].isna()] = -6.751

In [105]:
df_pandas.modo_de_audio.median()

1.0

In [106]:
df_pandas.loc[df_pandas['modo_de_audio'].isna()] = 1.0

In [107]:
df_pandas.discurso.median()

0.0541

In [108]:
df_pandas.loc[df_pandas['discurso'].isna()] = 0.0541

In [109]:
df_pandas.assinatura_de_tempo.median()

4.0

In [110]:
df_pandas.loc[df_pandas['assinatura_de_tempo'].isna()] = 4.0

In [111]:
df_pandas.isnull().sum()

nome_musica            0
popularidade           0
duracao                0
acustica               0
danceabilidade         0
energia                0
instrumentalidade      0
nota_musical           0
ao_vivo                0
volume                 0
modo_de_audio          0
discurso               0
ritmo                  0
assinatura_de_tempo    0
valencia_de_audio      0
dtype: int64

## VALIDAÇÃO DOS DADOS

In [112]:
df_pandas['nome_musica'] = df_pandas['nome_musica'].astype(str)

In [113]:
df_pandas['popularidade'] = df_pandas['popularidade'].astype(int)

In [114]:
df_pandas['duracao'] = df_pandas['duracao'].astype(int)

In [115]:
df_pandas['acustica'] = df_pandas['acustica'].astype(float)

In [116]:
df_pandas['danceabilidade'] = df_pandas['danceabilidade'].astype(float)

In [117]:
df_pandas['energia'] = df_pandas['energia'].astype(float)

In [118]:
df_pandas['instrumentalidade'] = df_pandas['instrumentalidade'].astype(str)

In [119]:
df_pandas['nota_musical'] = df_pandas['nota_musical'].astype(int)

In [120]:
df_pandas['popularidade'] = df_pandas['popularidade'].astype(int)

In [121]:
df_pandas['duracao'] = df_pandas['duracao'].astype(int)

In [122]:
df_pandas['nota_musical'] = df_pandas['nota_musical'].astype(int)

In [123]:
df_pandas['ao_vivo'] = df_pandas['ao_vivo'].astype(float)

In [124]:
df_pandas['volume'] = df_pandas['volume'].astype(float)

In [125]:
df_pandas['modo_de_audio'] = df_pandas['modo_de_audio'].astype(int)

In [126]:
df_pandas['discurso'] = df_pandas['discurso'].astype(float)

In [127]:
df_pandas['ritmo'] = df_pandas['ritmo'].astype(float)

In [128]:
df_pandas['assinatura_de_tempo'] = df_pandas['assinatura_de_tempo'].astype(float)

In [129]:
df_pandas['valencia_de_audio'] = df_pandas['valencia_de_audio'].astype(float)

In [130]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14920 entries, 0 to 14931
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   nome_musica          14920 non-null  object 
 1   popularidade         14920 non-null  int64  
 2   duracao              14920 non-null  int64  
 3   acustica             14920 non-null  float64
 4   danceabilidade       14920 non-null  float64
 5   energia              14920 non-null  float64
 6   instrumentalidade    14920 non-null  object 
 7   nota_musical         14920 non-null  int64  
 8   ao_vivo              14920 non-null  float64
 9   volume               14920 non-null  float64
 10  modo_de_audio        14920 non-null  int64  
 11  discurso             14920 non-null  float64
 12  ritmo                14920 non-null  float64
 13  assinatura_de_tempo  14920 non-null  float64
 14  valencia_de_audio    14920 non-null  float64
dtypes: float64(9), int64(4), object(2)
m

In [131]:
# criação do schema de validação dos dados

schema = pa.DataFrameSchema(
    columns = {
      'nome_musica':pa.Column(pa.String),
      'popularidade':pa.Column(pa.Int),
      'duracao':pa.Column(pa.Int),
      'acustica':pa.Column(pa.Float),
      'danceabilidade':pa.Column(pa.Float),
      'energia':pa.Column(pa.Float),
      'instrumentalidade':pa.Column(pa.String),
      'nota_musical':pa.Column(pa.Int),
      'ao_vivo':pa.Column(pa.Float),
      'volume':pa.Column(pa.Float),
      'modo_de_audio':pa.Column(pa.Int),
      'discurso':pa.Column(pa.Float),
      'ritmo':pa.Column(pa.Float),
      'assinatura_de_tempo':pa.Column(pa.Float),
      'valencia_de_audio':pa.Column(pa.Float)
    }
)

In [132]:
# validador do DF
schema.validate(df_pandas)

,nome_musica,popularidade,duracao,acustica,danceabilidade,energia,instrumentalidade,nota_musical,ao_vivo,volume,modo_de_audio,discurso,ritmo,assinatura_de_tempo,valencia_de_audio
0,Yeah!,79,250373,0.0183,0.894,0.791,0.0,2,0.0388,-4.699,1,0.1120,105.017998,4.0,0.583
1,Someday - Remastered,63,243333,0.0108,0.743,0.670,0.0702,1,0.2200,-4.547,0,0.0293,110.832001,4.0,0.934
2,Easy,58,184141,0.3550,0.847,0.703,3.950000000000001e-05,6,0.0938,-4.434,1,0.0746,112.044998,4.0,0.777
3,Do You Wanna Party,56,201960,0.1230,0.794,0.844,3.16e-06,11,0.0988,-3.222,1,0.0904,119.988998,4.0,0.794
4,La Media Vuelta,72,161040,0.2910,0.631,0.683,0.0,7,0.0994,-8.788,1,0.0344,97.959000,4.0,0.708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14927,Anywhere,77,205250,0.0162,0.717,0.872,1.99e-06,11,0.2050,-3.279,1,0.0357,120.026001,4.0,0.610
14928,Real Rich (feat. Gucci Mane),74,124525,0.0353,0.947,0.441,0.0,2,0.1110,-9.311,1,0.1350,125.010002,4.0,0.275
14929,I Found,58,274226,0.8770,0.280,0.317,0.007140000000000001,7,0.1080,-12.433,0,0.0468,186.643997,3.0,0.237
14930,3:15,76,167018,0.3850,0.651,0.654,0.0,1,0.1810,-5.481,1,0.0466,141.975006,4.0,0.539


## 9) ENCONTRE ALGUMAS ANÁLISES INTERESSANTES UTILIZANDO AS FUNÇÕES DO SPARK VISTAS EM AULA E LISTE PELO MENOS 5 DESSAS.

In [133]:
# convertendo do pandas para o spark
df_spark = spark.createDataFrame(df_pandas)

In [134]:
df_spark.show()

+--------------------+------------+-------+--------------------+-------------------+-------------------+--------------------+------------+-------------------+-------------------+-------------+--------------------+------------------+-------------------+-------------------+
|         nome_musica|popularidade|duracao|            acustica|     danceabilidade|            energia|   instrumentalidade|nota_musical|            ao_vivo|             volume|modo_de_audio|            discurso|             ritmo|assinatura_de_tempo|  valencia_de_audio|
+--------------------+------------+-------+--------------------+-------------------+-------------------+--------------------+------------+-------------------+-------------------+-------------+--------------------+------------------+-------------------+-------------------+
|               Yeah!|          79| 250373|0.018300000578165054| 0.8939999938011169| 0.7910000085830688|                 0.0|           2|0.03880000114440918| -4.698999881744385|   

In [135]:
# mostrando as 5 músicas mais tristes de acordo com a valência de áudio
df_spark.select(F.col('nome_musica'), F.col('valencia_de_audio')).orderBy(F.col('valencia_de_audio').asc()).show(5)

+--------------------+--------------------+
|         nome_musica|   valencia_de_audio|
+--------------------+--------------------+
|                Aura|                 0.0|
|Gina Rodriguez - ...|                 0.0|
|         Nijidion 01|0.023000000044703484|
|   Portugal (A Lift)|0.024299999698996544|
|   ...Familiar Place|0.027699999511241913|
+--------------------+--------------------+
only showing top 5 rows



In [136]:
# mostrando as 5 músicas mais felizes de acordo com a valência de áudio
df_spark.select(F.col('nome_musica'), F.col('valencia_de_audio')).orderBy(F.col('valencia_de_audio').desc()).show(5)

+--------------------+------------------+
|         nome_musica| valencia_de_audio|
+--------------------+------------------+
|  Sergio el Bailador| 0.984000027179718|
|Talking In Your S...|0.9819999933242798|
|           September|0.9819999933242798|
|      Quién Se Anima|0.9819999933242798|
|                  Ok|0.9810000061988831|
+--------------------+------------------+
only showing top 5 rows



In [137]:
# mostrando as 5 músicas com menos discurso
df_spark.select(F.col('nome_musica'), F.col('discurso')).orderBy(F.col('discurso').asc()).show(5)

+--------------------+-------------------+
|         nome_musica|           discurso|
+--------------------+-------------------+
|                Aura|                0.0|
|Gina Rodriguez - ...|                0.0|
|Woman - 2010 - Re...|0.02239999920129776|
|               Amigo|0.02280000038444996|
|Nothing's Gonna S...|0.02280000038444996|
+--------------------+-------------------+
only showing top 5 rows



In [138]:
# mostrando as 5 músicas com mais discurso (chances de ser poesias, entrevistas e audiolivros)
df_spark.select(F.col('nome_musica'), F.col('discurso')).orderBy(F.col('discurso').desc()).show(5)

+--------------------+------------------+
|         nome_musica|          discurso|
+--------------------+------------------+
|       Space Program|0.9409999847412109|
|Racism And Injustice|0.9399999976158142|
|Parents' Divorce ...|0.9359999895095825|
|               blame|0.9150000214576721|
|        high.flying.| 0.906000018119812|
+--------------------+------------------+
only showing top 5 rows



In [139]:
# mostrando as 5 músicas mais dançáveis
df_spark.select(F.col('nome_musica'), F.col('danceabilidade')).orderBy(F.col('danceabilidade').desc()).show(5)

+------------+------------------+
| nome_musica|    danceabilidade|
+------------+------------------+
|   Walk Away|0.9869999885559082|
|   HOLLYWOOD|0.9810000061988831|
|Ice Ice Baby|0.9800000190734863|
|Ice Ice Baby|0.9779999852180481|
|Funky Friday|0.9750000238418579|
+------------+------------------+
only showing top 5 rows

